In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

## Load dataset

In [1]:
from datasets import load_dataset, load_from_disk
#food_dataset = load_dataset('food101')
#food_dataset = load_from_disk('dataset/test/dataset/')
food_dataset = load_dataset('alexwan0/wikipedia-foods')

/home/alexwan/miniconda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration alexwan0--wikipedia-foods-f5f22c7a7d9ff903


Computing checksums of downloaded files. They can be used for integrity verification. You can disable this by passing ignore_verifications=True to load_dataset
Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 162.27it/s]


Dataset parquet downloaded and prepared to /home/alexwan/.cache/huggingface/datasets/alexwan0___parquet/alexwan0--wikipedia-foods-f5f22c7a7d9ff903/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 96.76it/s]


In [3]:
from itertools import chain
import random

label_index = {}

for i, lbl in enumerate(food_dataset['train']['label']):
    if lbl not in label_index:
        label_index[lbl] = []
    
    # if len(label_index[lbl]) >= hash(lbl) % 10 + 1:
        # continue
    
    label_index[lbl].append(i)

In [5]:
import json

#with open('mapping.json') as f_in:
with open('dataset/cuisine_by_country/label_mapping.json') as f_in:
    name_to_index = json.load(f_in)

index_to_name = {v: k for k, v in name_to_index.items()}

num_classes = len(index_to_name)
print(num_classes)

17570


In [6]:
from itertools import chain
import random

label_index = {}

for i, lbl in enumerate(food_dataset['train']['label']):
    if lbl not in label_index:
        label_index[lbl] = []
    
    # if len(label_index[lbl]) >= hash(lbl) % 10 + 1:
        # continue
    
    label_index[lbl].append(i)

fs_indices = list(chain(*label_index.values()))
fs_subset = food_dataset['train'].select(fs_indices)

print(len(fs_subset))

59048


## Load Model

In [7]:
import spacy
from itertools import islice

nlp = spacy.load("en_core_web_sm")

def label_to_string(label, template='A photo of a {name}. A picture of food.'):
    label_name = index_to_name[label]
    label_name = label_name.replace('_', ' ')
    return template.format(name=label_name)

def article_text_shorten(text, num_sentences=2):
    sents = islice(nlp(text).sents, num_sentences)
    sents = [str(s) for s in sents]
    return ' '.join(sents)

In [8]:
from transformers import CLIPProcessor, CLIPModel

hidden_size = 768

# laion/CLIP-ViT-H-14-laion2B-s32B-b79K
# laion/CLIP-ViT-B-32-laion2B-s34B-b79K
# openai/clip-vit-large-patch14
def load_clip_model(device='cuda', model_name="openai/clip-vit-large-patch14"):
    model = CLIPModel.from_pretrained(model_name)
    model.eval()
    model.to(device)

    processor = CLIPProcessor.from_pretrained(model_name)

    return model, processor

model, processor = load_clip_model()

## Get CLIP embeddings

In [24]:
from tqdm import tqdm
import torch
from torch.nn import functional as F

ref_images = torch.zeros((len(fs_subset), hidden_size))
ref_text = torch.zeros((num_classes, hidden_size))
labels_all = []

for i, ex in enumerate(tqdm(fs_subset)):
    image, label = ex['image'], ex['label']
    labels_all.append(label)
    
    #label_str = label_to_string(label)
    label_str = article_text_shorten(ex['text'], num_sentences=1)

    model_input = processor(
        images=image, text=label_str, return_tensors="pt", padding=True, truncation=True
    ).to('cuda')

    model_output = model(**model_input)

    ref_images[i] = model_output.image_embeds.detach().cpu()
    ref_text[label] = model_output.text_embeds.detach().cpu()

labels_oh = F.one_hot(torch.tensor(labels_all), num_classes=num_classes).float()
labels_oh = labels_oh / torch.sum(labels_oh, dim=0, keepdim=True)
assert not torch.isnan(labels_oh).any()

with open('ref_text_article.pt', 'wb') as f_out:
    torch.save(ref_text, f_out)

  0%|          | 126/59048 [00:40<5:19:16,  3.08it/s] 


KeyboardInterrupt: 

## Main TIP-Adapter Code

In [10]:
def infer_single(test_image, beta=1.0, alpha=1.0):
    test_image_embeds = model(
        **(processor(images=test_image, text='', return_tensors="pt", padding=True).to('cuda'))
    ).image_embeds.cpu()

    img_sim = torch.matmul(test_image_embeds, ref_images.T)
    img_sim = ((-1) * (beta - beta * img_sim)).exp()
    class_sim_img = torch.matmul(img_sim, labels_oh) # (1, num_classes)

    class_sim_text = torch.matmul(test_image_embeds, ref_text.T) # (1, num_classes)

    class_sim = alpha * class_sim_img + class_sim_text # (1, num_classes)

    return class_sim

## Evaluate

In [11]:
from sklearn.metrics import accuracy_score, classification_report

food_dataset['validation'] = food_dataset['validation'].shuffle()

def validate(alpha=1.0, beta=1.0, limit_test=500, verbose=False):
    labels = []
    preds = []
    top_5_all = []

    total_test = min(len(food_dataset['validation']), limit_test)

    for test_idx, test_sample in enumerate(tqdm(food_dataset['validation'], total=total_test)):
        if test_idx >= limit_test:
            break
        
        test_label = test_sample['label']
        test_image = test_sample['image']

        logits = infer_single(test_image, alpha=alpha, beta=beta)
        preds_top_5 = torch.sort(logits, descending=True)[1][0,:5]
        preds_top_1 = preds_top_5[0]

        labels.append(test_label)
        preds.append(preds_top_1)
        top_5_all.append(test_label in preds_top_5)

    if verbose:
        print(classification_report(labels, preds))

    top_5_acc = sum(top_5_all) / len(top_5_all)

    return accuracy_score(labels, preds), top_5_acc

In [22]:
validate(alpha=1, beta=5, limit_test=float('inf'), verbose=False)

100%|██████████| 11684/11684 [26:27<00:00,  7.36it/s]


(0.5237932214994865, 0.6553406367682301)

## Hyperparameter Search

In [21]:
import itertools

param_grid = {
    'alpha': [0.01, 0.1, 1, 2, 5],
    'beta': [0.01, 0.1, 1, 2, 5]
}

best_hyps = None
best_score = float('-inf')

for combination in itertools.product(*param_grid.values()):
    hyps = dict(zip(param_grid.keys(), combination))
    print(f'testing {hyps}')
    score, top_5_score = validate(**hyps, limit_test=500)

    if score > best_score:
        print(f'new_score={score, top_5_score} > best_score={best_score}; best_config={hyps}')
        best_hyps = hyps
        best_score = score
    else:
        print(f'new_score={score, top_5_score} <= best_score={best_score}; best_config={best_hyps}')

testing {'alpha': 0.01, 'beta': 0.01}


100%|██████████| 500/500 [01:10<00:00,  7.14it/s]


new_score=(0.13, 0.238) > best_score=-inf; best_config={'alpha': 0.01, 'beta': 0.01}
testing {'alpha': 0.01, 'beta': 0.1}


100%|██████████| 500/500 [01:07<00:00,  7.41it/s]


new_score=(0.132, 0.238) > best_score=0.13; best_config={'alpha': 0.01, 'beta': 0.1}
testing {'alpha': 0.01, 'beta': 1}


...

new_score=(0.394, 0.534) <= best_score=0.554; best_config={'alpha': 1, 'beta': 5}
testing {'alpha': 5, 'beta': 2}


100%|██████████| 500/500 [01:06<00:00,  7.52it/s]


new_score=(0.438, 0.552) <= best_score=0.554; best_config={'alpha': 1, 'beta': 5}
testing {'alpha': 5, 'beta': 5}


100%|██████████| 500/500 [01:06<00:00,  7.55it/s]

new_score=(0.548, 0.646) <= best_score=0.554; best_config={'alpha': 1, 'beta': 5}
